You can run the code by the following command

`python panorama.py`

### Harris Corner Detector

Harris is implemented using the following steps
1. X and Y derivatives of the image is calculated using sobel in OpenCV (This is done by specifying the windows size which moves on image pixels and calculates derivatives in it)
2. Covariance of the Matrix is calculated by defining $I_{x}^{2}$, $I_{y}^{2}$ and $I_{x}I_{y}$
3. Now we convolve the structure tensor with a Gaussian filter to smooth the results
4. Now we should calculate the following equation which gives us the Harris responce we need: $ Det(H) / Trace(H) = det(A) - kTrace^2(A)$ 
5. At last we should find the corners in the results above a specific threshold
6. As it is noticeable above, there are several hyper parameters that needs to be defined for best results which are k, window size and threshold. To find the best combination of these I wrote an experiment of different variables of each and exported the image of the results with the corresponding values of it. Some of the outputs are visible below:
    - For K=0.04, windowSize=3 and threshold=0.99
    <br>
    <img src="./harris_results/harris_k0.04_window3_threshold0.99.jpg" width="400" height="200">
    - For K=0.04, windowSize=3 and threshold=0.99999999
    <br>
    <img src="./harris_results/harris_k0.04_window3_threshold0.99999999.jpg" width="400" height="200">
    <br>
By running the code a total of 440 images with different hyper parameters will be generated.


Bellow is the function used for detecting Harris corners

In [1]:
def harris_corner_detector(image, k=0.4, window_size=3, threshold=0.8):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    height, width = gray.shape
    dx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=window_size)
    dy = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=window_size)
    Ixx = dx**2
    Ixy = dy*dx
    Iyy = dy**2

    # Define the size of the Gaussian filter
    ksize = 5

    # Define the standard deviation of the Gaussian filter
    sigma = 1.5

    # Convolve the structure tensor elements with a Gaussian filter
    Ixx = cv2.GaussianBlur(Ixx, (ksize, ksize), sigma)
    Iyy = cv2.GaussianBlur(Iyy, (ksize, ksize), sigma)
    Ixy = cv2.GaussianBlur(Ixy, (ksize, ksize), sigma)

    corner_response = np.zeros_like(gray)

    for i in range(height):
        for j in range(width):
            Sxx = np.sum(Ixx[i-window_size//2:i+window_size//2+1, j-window_size//2:j+window_size//2+1])
            Syy = np.sum(Iyy[i-window_size//2:i+window_size//2+1, j-window_size//2:j+window_size//2+1])
            Sxy = np.sum(Ixy[i-window_size//2:i+window_size//2+1, j-window_size//2:j+window_size//2+1])
            det = Sxx * Syy - Sxy**2
            trace = Sxx + Syy
            corner_response[i, j] = det - k * trace**2
    corner_response = np.where(corner_response >= threshold * corner_response.max(), corner_response, 0)
    print(threshold * corner_response.max())
    for i in range(height):
        for j in range(width):
            if corner_response[i, j] != 0:
                cv2.circle(image, (j, i), 5, (0, 0, 255), -1)
    return image